<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/master/fer2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *

%matplotlib inline

In [16]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/cs230 project/collab/fer2013/fer2013.csv')

#print('Number of samples in the dataset: ', data.shape[0])
# Transform images from strings to lists of integers. TODO: use an array cast
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
emotion_cat = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

In [0]:
data_train = data[data['Usage']=='Training']
#print('Number samples in the training dataset: ', data_train.shape[0])

data_dev = data[data['Usage']=='PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])

# Retrieve train input and target
X_train, y_train = data_train['pixels'].tolist(), data_train['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_train = np.array(X_train, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max (the maximum pixel intensity is 255)
X_train = X_train/255.0

# Retrieve dev input and target
X_dev, y_dev = data_dev['pixels'].tolist(), data_dev['emotion'].values
X_dev = np.array(X_dev, dtype='float32').reshape(-1,48,48,1)
X_dev = X_dev/255.0

In [0]:
#Baseline model
model = tf.keras.models.Sequential([
    InputLayer(input_shape=(48,48,1),name="input"),
    Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv1"),
    Dropout(0.25),
    Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv2"),
    Dropout(0.25),
    MaxPool2D(pool_size=(2,2),name="maxpool1"),
    Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv3"),
    Dropout(0.25),
    Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv4"),
    Dropout(0.25),
    Flatten(),
    Dense(1024,input_shape=(24*24*64,1),activation='relu',name='fc1'),
    Dense(7,input_shape=(1024,1),activation='softmax',name='fc-softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
model.fit(X_train,y_train,batch_size=32,epochs=5,validation_data=(X_dev, y_dev))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 26s 897us/sample - loss: 0.5994 - acc: 0.7847 - val_loss: 1.3848 - val_acc: 0.5311
Epoch 2/5
28709/28709 [==============================] - 26s 896us/sample - loss: 0.3668 - acc: 0.8747 - val_loss: 1.5980 - val_acc: 0.5308
Epoch 3/5
28709/28709 [==============================] - 26s 891us/sample - loss: 0.2380 - acc: 0.9213 - val_loss: 1.7969 - val_acc: 0.5216
Epoch 4/5
28709/28709 [==============================] - 26s 891us/sample - loss: 0.1849 - acc: 0.9428 - val_loss: 2.0679 - val_acc: 0.5130
Epoch 5/5
28709/28709 [==============================] - 26s 889us/sample - loss: 0.1480 - acc: 0.9549 - val_loss: 2.1488 - val_acc: 0.5074


In [126]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/cs230 project/webcam images',
        target_size=(48, 48),
        color_mode='grayscale',
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

nb_samples = len(test_generator.filenames)

y_true = test_generator.classes
#evaluate_generator
y_pred = model.predict_generator(test_generator, steps=nb_samples).argmax(axis=1)

Found 84 images belonging to 7 classes.


In [127]:
y_true-y_pred

array([-6, -6, -6, -2, -2, -6, -4,  0, -2, -2, -2, -6, -6, -6, -4, -6, -6,
       -3, -6, -6, -3, -5,  1, -2, -3, -3, -3,  1, -1,  1, -3, -3, -5, -3,
       -5, -3, -5, -3, -3, -5, -5, -5, -5, -5, -5, -2, -3,  1, -5, -2,  0,
        0,  0,  0,  0, -3,  0,  0,  0,  0, -3,  1, -1,  0,  0,  0,  0,  2,
       -2, -2, -2, -2,  2,  2,  2,  2,  2,  0,  0,  3,  0,  0,  0,  0])

In [128]:
test_generator.filenames[0]

'0 angry/angry0000.png'